### Week 3 Warmup Exercise

For this warmup exercise, we will be creating a Postgres database table and loading it with the standard iris dataset.  Once loaded, there are some analysis questions for you answer.

#### Import stuff

In [1]:
import psycopg2
import pandas as pd

#### Establish a connection to Postgres

In [2]:
# establish connection to progress and d=create the database
conn = psycopg2.connect(user='postgres')
conn.autocommit = True
cur = conn.cursor()

In [3]:
#create the iris_db
cur.execute("drop database if exists iris_db;")
cur.execute("create database iris_db;")

In [4]:
# closing this connection and we will re-establish it to the iris_db
conn.close()

##### Create the iris database table.  
Notice the drop command first... If you plan on rerunning this multiple times, you need to remove the existing iris database first

In [5]:
# now connect to just the iris_db
conn = psycopg2.connect(database ='iris_db', user='postgres')
cur = conn.cursor()

In [6]:
# drop the database if it exists
cur.execute("drop table if exists iris;")

In [7]:
# create table
cur.execute("""
    create table iris(
    sepal_length numeric,
    sepal_width numeric,
    petal_length numeric,
    petal_width numeric,
    species text)
""")
conn.commit;

In [8]:
# let's get a listing of all the tables in the database
cur.execute("select table_schema, table_name from information_schema.tables \
    where table_schema = 'public' order by table_schema, table_name;")
cur.fetchall()

[('public', 'iris')]

In [9]:
# let's look at the structure of the iris table
cur.execute("select column_name,data_type from information_schema.columns \
            where table_schema = 'public' and table_name='iris'")
cur.fetchall()

[('sepal_length', 'numeric'),
 ('sepal_width', 'numeric'),
 ('petal_length', 'numeric'),
 ('petal_width', 'numeric'),
 ('species', 'text')]

#### Load the iris dataset

In [10]:
# using a pandas dataframe for this, because it's easy
url ='https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
iris_data = pd.read_csv(url)

In [11]:
# it's always a good idea to verify that you have data in your dataframe
iris_data

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [12]:
# iterate over each row in the dataframe
for index, row in iris_data.iterrows():
    # assign the data to varibales
    s_l,s_w,p_l,p_w,s = row['sepal_length'],row['sepal_width'],row['petal_length'],row['petal_width'],row['species']
    
    #build an insert statement
    insert_str = "insert into iris values (%s, %s, %s, %s, '%s')" %(s_l,s_w,p_l,p_w,s)
    
    #insert the row into the database
    cur.execute(insert_str)

#Very important! save all tha data into the database
conn.commit;   

##### Verify that the database has data
We know that the iris dataset has 150 rows in it, so we should see 150 in this table

In [13]:
# let's look at the structure of the iris table
cur.execute("select count(*) from iris")
cur.fetchall()

[(150,)]

### Warmup Exercise
Using the `iris` table that you just created, answer the following questions.

1. What is the average sepal_length, sepal_width, petal_length and petal_width of all iris?

2. Expand upon question 1) and provide the same metrics per species of iris.

3. What is the range petal_length and petal_width for each species of iris?

4. Repeat question 3) for sepal_length and sepal_width.

In [19]:
#close the db connection
#conn.close()